In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import confusion_matrix, classification_report

import os

In [2]:
tf.config.list_physical_devices(
    device_type=None
)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
dataset_path = "../DataSet/Indian Bird Species (25 Classes)/"

In [4]:
os.listdir(dataset_path)

['Asian Green Bee-Eater',
 'Brown-Headed Barbet',
 'Cattle Egret',
 'Common Kingfisher',
 'Common Myna',
 'Common Rosefinch',
 'Common Tailorbird',
 'Coppersmith Barbet',
 'Forest Wagtail',
 'Gray Wagtail',
 'Hoopoe',
 'House Crow',
 'Indian Grey Hornbill',
 'Indian Peacock',
 'Indian Pitta',
 'Indian Roller',
 'Jungle Babbler',
 'Northern Lapwing',
 'Red-Wattled Lapwing',
 'Ruddy Shelduck',
 'Rufous Treepie',
 'Sarus Crane',
 'White Wagtail',
 'White-Breasted Kingfisher',
 'White-Breasted Waterhen']

In [ ]:
datagen = ImageDataGenerator(rescale= 1./255., validation_split = 0.10, rotation_range = 15, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.1, zoom_range=0.1, horizontal_flip=True, fill_mode = 'nearest')
train_set = datagen.flow_from_directory(directory = dataset_path, class_mode = 'categorical', target_size=(720, 480), color_mode="rgb", subset = 'training', batch_size = 4)
validation_set = datagen.flow_from_directory(directory = dataset_path, class_mode = 'categorical', target_size=(720, 480), color_mode="rgb", subset = 'validation', batch_size = 4)

In [ ]:
image_shape = (720, 480, 3)

In [ ]:
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=image_shape,
                   pooling='max',classes=25,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)

resnet_model.add(Flatten())
resnet_model.add(Dense(128, activation='relu'))
resnet_model.add(Dense(25, activation='softmax'))

resnet_model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
resnet_model.summary()

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=12)

In [ ]:
resnet_model.fit(train_set, epochs=60, validation_data=validation_set, callbacks=[early_stop])

In [ ]:
history = pd.DataFrame(resnet_model.history.history)

In [ ]:
history[["loss", "val_loss"]].plot()
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.title("Loss vs Epochs")
plt.show()

In [ ]:
history[['accuracy', 'val_accuracy']].plot()
plt.xlabel("Number of Epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy vs Epochs")
plt.show()

In [ ]:
predictions = resnet_model.predict(validation_set)

In [ ]:
predictions = np.argmax(predictions, axis=1)

In [ ]:
print(classification_report(validation_set.classes, predictions))

In [ ]:
confusion_matrix(validation_set.classes, predictions)

In [ ]:
resnet_model.save("IndianBirdsClassificationModel.h5")